# Strategy 3: Finetune hyperparameters
## In this improvement strategy, we use Weights & Biases sweep tool to automate hyperparameter search, within fixed space that we define. We would first have to define the search space by creating a configuration of the parameters we will be tweaking and their domain, along with the metric we are trying to minimise, which is eval/loss.

## https://api.wandb.ai/links/alvinjoseph/84ncenjd

## The above report shows a sweep with 20 runs, with various paramters being tweaked. From the results, we can observe that lora_alpha has the highest correlation (negative) with eval/loss and having a value of 16 seems to be the best for minimising the error. We also update the values for weight decay and warmup ratio.

# Strategy 3 Part 1

In [1]:
# Installing required packages
!pip install wandb
!pip install -U -q peft==0.6.2 transformers==4.35.2 datasets==2.15.0 bitsandbytes==0.41.2.post2 trl==0.7.4 accelerate==0.24.1 scipy==1.12.0 wandb==0.16.5 coloredlogs==15.0.1

In [2]:
# Load required packages

from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from datasets import load_dataset, Dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import torch

import pickle


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
# load SUTD QA dataset from step 1
with open('sutd_qa_dataset_strategy1.pkl', 'rb') as f:
    sutd_qa_dataset = pickle.load(f)

In [4]:
# split data into traing and test set, 160 instances for train, rest for test
sutd_qa_dataset = sutd_qa_dataset.train_test_split(train_size=160, shuffle=False)

In [5]:
# check schema and number of instances
sutd_qa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 160
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 46
    })
})

In [6]:
# inspect first instance
sutd_qa_dataset["train"][0]

{'question': 'How do I access the digital library?',
 'answer': ' You can access the digital library by logging into the SUTD portal and clicking on the "Library" tab. From there, you can access a variety of online resources, including e-books, academic journals, and other publications. Additionally, you can use the library\'s online catalog to search for physical resources and request items for delivery to the library.'}

In [7]:
# QUESTION: create a formating function 'formatting_func' which takes an example from your QA dataset as input and outputs 
# a dictionary with the key "text" and as value a text prompt with the following format:
# ### USER: {question from example goes here}
# ### ASSISTANT: {answer from example goes here}


#--- ADD YOUR SOLUTION HERE (10 points)---
def formatting_func(example):
    formatted_text = f"### USER: {example['question']}\n### ASSISTANT: {example['answer']}"
    return {"text": formatted_text}


#----------------------------------------


In [8]:
# apply formatting function to data set
formatted_dataset = sutd_qa_dataset.map(formatting_func)

Map: 100%|██████████| 46/46 [00:00<00:00, 7791.07 examples/s]


In [9]:
# check formatted prompt
formatted_dataset["train"]["text"][0]

# Note: you should see something like this (not necessary the same prompt but same format)
# '### USER: What are some of the best places to eat near the SUTD campus?\n### ASSISTANT: There are several great dining options near the SUTD campus. 
# One popular spot is the Changi Business Park Food Court, ...


'### USER: How do I access the digital library?\n### ASSISTANT:  You can access the digital library by logging into the SUTD portal and clicking on the "Library" tab. From there, you can access a variety of online resources, including e-books, academic journals, and other publications. Additionally, you can use the library\'s online catalog to search for physical resources and request items for delivery to the library.'

In [10]:
# model id of base model
model_id = "NousResearch/Nous-Hermes-llama-2-7b"

# model id for our finetuned model
new_model = "nous-hermes-7b-qlora-sutd-qa"

# config for model quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    use_nested_quant = False
)

# Load the entire model on the GPU 0
device_map = {"": 0}


In [11]:
# Login to wandb
import wandb
wandb.login()

wandb: Currently logged in as: alvinjoseph. Use `wandb login --relogin` to force relogin


True

In [12]:
# sweep config
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'eval/loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'learning_rate': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
    },
    'weight_decay': {
        'values': [0, 0.001, 0.01]
    },
    'max_grad_norm': {
        'values': [0.1, 0.3, 1.0]
    },
    'lora_alpha': {
        'values': [4, 8, 16]
    },
    'r': {
        'values': [4, 8, 16]
    },
    'warmup_ratio': {
        'values': [0.01, 0.03, 0.1]
    }
}


sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="llama-finetune-sweep")

Create sweep with ID: lnbkc5fm
Sweep URL: https://wandb.ai/alvinjoseph/llama-finetune-sweep/sweeps/lnbkc5fm


In [14]:
import json 
import gc

def train(config=None):
    with wandb.init(config=config):
        model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map=device_map)
        model.config.use_cache = False
        model.config.pretraining_tp = 1
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"
        config = wandb.config
        lora_config = LoraConfig(
            lora_alpha=config.lora_alpha,
            r=config.r,
            lora_dropout=0.1,
            bias="none",
            task_type="CAUSAL_LM",
        )
        output_dir = "./results"
        per_device_train_batch_size = 1
        gradient_accumulation_steps = 1
        optim = "paged_adamw_32bit"
        save_steps = 10
        logging_steps = 10
        learning_rate = 2e-4
        weight_decay = config.weight_decay
        max_grad_norm = config.max_grad_norm
        num_train_epochs = 1
        warmup_ratio = config.warmup_ratio
        lr_scheduler_type = "cosine"
        packing = False
        max_seq_length = None

        training_arguments = TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            optim=optim,
            save_steps=save_steps,
            logging_steps=logging_steps,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            max_grad_norm=max_grad_norm,
            num_train_epochs=num_train_epochs,
            warmup_ratio=warmup_ratio,
            lr_scheduler_type=lr_scheduler_type,
            report_to="wandb"
            
        )

        trainer = SFTTrainer(
            model=model,
            args=training_arguments,
            train_dataset=formatted_dataset["train"],
            eval_dataset=formatted_dataset["test"],
            peft_config=lora_config,
            dataset_text_field="text",
            packing=packing,
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
        )

        trainer.train()
        text = trainer.evaluate()
        print("output")
        print(text)
        del trainer
        del model
        del tokenizer
    
        gc.collect()
        torch.cuda.empty_cache()

wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: 3hq244sz with config:
wandb: 	learning_rate: 0.023460014383040777
wandb: 	lora_alpha: 4
wandb: 	max_grad_norm: 1
wandb: 	r: 8
wandb: 	warmup_ratio: 0.03
wandb: 	weight_decay: 0.01


/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:194: User

Step,Training Loss
10,1.836200
20,1.674000
30,1.638500
40,1.488200
50,1.418900
60,1.298700
70,1.009500
80,1.325500
90,1.075300
100,1.224400


output
{'eval_loss': 1.1717647314071655, 'eval_runtime': 10.3696, 'eval_samples_per_second': 4.436, 'eval_steps_per_second': 0.579, 'epoch': 1.0}


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇████
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇████
train/learning_rate,███▇▇▆▅▅▄▃▃▂▂▁▁▁
train/loss,█▇▆▅▄▃▁▄▂▃▂▂▂▂▂▃
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


...

wandb: Agent Starting Run: sg4jmlfw with config:
wandb: 	learning_rate: 0.04421024821112551
wandb: 	lora_alpha: 16
wandb: 	max_grad_norm: 1
wandb: 	r: 16
wandb: 	warmup_ratio: 0.01
wandb: 	weight_decay: 0.01


/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:194: User

Step,Training Loss
10,1.779900
20,1.501600
30,1.375600
40,1.369000
50,1.340300
60,1.174000
70,0.864300
80,1.213200
90,0.934200
100,1.108900


output
{'eval_loss': 1.068800687789917, 'eval_runtime': 10.3891, 'eval_samples_per_second': 4.428, 'eval_steps_per_second': 0.578, 'epoch': 1.0}


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇████
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇████
train/learning_rate,███▇▇▆▅▅▄▃▃▂▂▁▁▁
train/loss,█▆▅▅▅▃▁▄▂▃▂▂▂▂▂▃
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


### This concludes the first part of Strategy 3. Continue with the next part.